In [1]:
import pdfplumber
import pandas as pd
import re

In [2]:
pdf = pdfplumber.open("data/jpn2022.pdf")

## number of issues

In [3]:
page_jpn_issue = pdf.pages[11]


In [4]:
jpn_issue = page_jpn_issue.extract_table()
df_jpn_issue = pd.DataFrame(jpn_issue[2:], columns=jpn_issue[1])
df_jpn_issue

,None,数次往復,一往復・限定,小計,外交,公用,小計,None,None
0,昭和21年,－,－,8,－,－,－,8,-
1,22,－,－,11,－,1,1,12,-
2,23,－,－,163,－,－,－,163,-
3,24,－,－,857,－,－,－,857,-
4,25,－,－,"3,291",－,－,－,"3,291",-
...,...,...,...,...,...,...,...,...,...
72,30,"4,182,207",None,"4,182,207","2,608","25,402","28,010","4,210,217","18,954,031"
73,31/令和元年,"4,365,290",None,"4,365,290","2,613","23,857","26,470","4,391,760","20,080,669"
74,2,"1,234,928",None,"1,234,928","2,024","11,414","13,438","1,248,366","3,174,219"
75,3,"513,943",None,"513,943","2,395","11,485","13,880","527,823","512,244"


In [5]:
df_jpn_issue.columns = ['Jpn_year', 'Ordinary_Multiple', 'Ordinary_One', 'Ordinary', 'Diplomatic', 'Official', 'Official_Diplomatic', 'total', 'num_departures']
df_jpn_issue.tail(3)

,Jpn_year,Ordinary_Multiple,Ordinary_One,Ordinary,Diplomatic,Official,Official_Diplomatic,total,num_departures
74,2,"1,234,928",None,"1,234,928","2,024","11,414","13,438","1,248,366","3,174,219"
75,3,"513,943",None,"513,943","2,395","11,485","13,880","527,823","512,244"
76,4,"1,218,692",None,"1,218,692","2,603","17,640","20,243","1,238,935","2,771,634"


In [6]:
#Western calendar
df_jpn_issue['year'] = range(1946, 1946 + len(df_jpn_issue))


In [7]:
df_jpn_issue = df_jpn_issue.replace(['－', None, '-'], '-')
df_jpn_issue

,Jpn_year,Ordinary_Multiple,Ordinary_One,Ordinary,Diplomatic,Official,Official_Diplomatic,total,num_departures,year
0,昭和21年,-,-,8,-,-,-,8,-,1946
1,22,-,-,11,-,1,1,12,-,1947
2,23,-,-,163,-,-,-,163,-,1948
3,24,-,-,857,-,-,-,857,-,1949
4,25,-,-,"3,291",-,-,-,"3,291",-,1950
...,...,...,...,...,...,...,...,...,...,...
72,30,"4,182,207",-,"4,182,207","2,608","25,402","28,010","4,210,217","18,954,031",2018
73,31/令和元年,"4,365,290",-,"4,365,290","2,613","23,857","26,470","4,391,760","20,080,669",2019
74,2,"1,234,928",-,"1,234,928","2,024","11,414","13,438","1,248,366","3,174,219",2020
75,3,"513,943",-,"513,943","2,395","11,485","13,880","527,823","512,244",2021


## Number of valid passports

In [8]:
pdf = pdfplumber.open("data/jpn2021.pdf")
page_jpn_valid = pdf.pages[10]
jpn_valid = page_jpn_valid.extract_table()
pd_jpn_valid = pd.DataFrame(jpn_valid[1:], columns=jpn_valid[0])
pd_jpn_valid.head(3)

,都道府県名,5年旅券,10年旅券,合計
0,北海道,"155,232","474,515","629,747"
1,青森,"26,202","63,499","89,701"
2,岩手,"28,877","67,232","96,109"


In [9]:
2018-22

1996

In [10]:
years = [2018,2019,2020,2021,2022]
dfs = []
for year in years:
    pdf = pdfplumber.open(f"data/jpn{year}.pdf")
    page_jpn_valid = pdf.pages[10]
    jpn_valid = page_jpn_valid.extract_table()
    pd_jpn_valid = pd.DataFrame(jpn_valid[1:], columns=jpn_valid[0])
    pd_jpn_valid.rename(columns={"合計": str(year)}, inplace=True)
    pd_jpn_valid.rename(columns={"5年旅券": f"5_{year}", "10年旅券": f"10_{year}"}, inplace=True)
    dfs.append(pd_jpn_valid)
result_df = pd.concat(dfs, axis=1)

result_df.set_index("都道府県名", inplace=True)

result_df.index = result_df.index.map(lambda x: x[0])

result_df_2018to2022 = result_df
result_df_2018to2022

,5_2018,10_2018,2018,5_2019,10_2019,2019,5_2020,10_2020,2020,5_2021,10_2021,2021,5_2022,10_2022,2022
都道府県名,,,,,,,,,,,,,,,
北海道,"208,300","567,634","775,934","218,887","572,357","791,244","194,208","528,757","722,965","155,232","474,515","629,747","118,610","427,793","546,403"
青森,"36,632","78,073","114,705","37,984","77,698","115,682","33,143","70,901","104,044","26,202","63,499","89,701","19,933","56,248","76,181"
岩手,"41,302","79,724","121,026","42,086","80,494","122,580","36,020","73,852","109,872","28,877","67,232","96,109","21,868","60,365","82,233"
宮城,"98,144","261,822","359,966","101,047","263,223","364,270","87,667","244,487","332,154","70,188","223,013","293,201","53,055","198,919","251,974"
秋田,"27,229","61,974","89,203","28,305","61,017","89,322","24,633","55,347","79,980","19,319","49,272","68,591","14,433","43,445","57,878"
山形,"44,698","88,795","133,493","44,889","88,679","133,568","38,318","81,127","119,445","29,815","72,623","102,438","21,922","64,489","86,411"
福島,"80,008","177,721","257,729","79,817","175,730","255,547","68,045","160,576","228,621","51,792","145,434","197,226","37,953","128,340","166,293"
茨城,"145,786","420,889","566,675","149,998","412,861","562,859","131,785","374,155","505,940","104,546","334,193","438,739","80,070","301,163","381,233"
栃木,"95,673","257,308","352,981","98,427","253,879","352,306","85,902","231,699","317,601","67,926","206,000","273,926","52,043","184,497","236,540"


### 2015-17

In [11]:


def create_prefecture_name(office_name):
    prefectures = {
        "埼玉県庁": "埼玉県", "川越分室": "埼玉県", "春日部分室": "埼玉県",
        "千葉県庁": "千葉県", "東葛飾分室": "千葉県",
        "東京都庁": "東京都", "有楽町分室": "東京都", "立川分室": "東京都", "池袋分室": "東京都",
        "神奈川県庁": "神奈川県", "厚木分室": "神奈川県", "川崎分室": "神奈川県",
        "愛知県庁": "愛知県", "豊橋分室": "愛知県",
        "大阪府庁": "大阪府", "阿倍野分室": "大阪府", "りんくう分室": "大阪府",
        "兵庫県庁": "兵庫県", "尼崎分室": "兵庫県",
        "福岡県庁": "福岡県", "北九州分室": "福岡県",
    }
    for key, value in prefectures.items():
        if key in office_name:
            return value
    return office_name.rstrip("庁")

years = [2015, 2016, 2017]
dfs = []
for year in years:
    pdf = pdfplumber.open(f"data/jpn{year}.pdf")
    page_jpn_valid = pdf.pages[10]
    jpn_valid = page_jpn_valid.extract_table()
    pd_jpn_valid = pd.DataFrame(jpn_valid[2:], columns=jpn_valid[1])
    
    pd_jpn_valid["5年旅券"] = pd_jpn_valid["5年旅券"].str.replace(",", "").astype(int)
    pd_jpn_valid["10年旅券"] = pd_jpn_valid["10年旅券"].str.replace(",", "").astype(int)
    pd_jpn_valid["合計"] = pd_jpn_valid["合計"].str.replace(",", "").astype(int)
    
    pd_jpn_valid.rename(columns={"合計": str(year)}, inplace=True)
    pd_jpn_valid.rename(columns={"5年旅券": f"5_{year}", "10年旅券": f"10_{year}"}, inplace=True)
    pd_jpn_valid["事務所名"] = pd_jpn_valid["事務所名"].apply(create_prefecture_name)
    dfs.append(pd_jpn_valid)

result_df = pd.concat(dfs, axis=1)

result_df.set_index("事務所名", inplace=True)

result_df.index = result_df.index.map(lambda x: x[0])
pd.set_option('display.max_rows', None)
result_df

result_df = result_df.groupby(result_df.index).sum()




# 「都」「府」「県」をとる（北海道はそのまま）
result_df.index = result_df.index.str.rstrip("都府県")

# 「外務省」の行の値を「外務省・在外公館」に変更
result_df.rename(index={"外務省": "外務省・在外公館"}, inplace=True)
result_df.rename(index={"京": "京都"}, inplace=True)
# 列名「事務所名」を「都道府県名」に改める
result_df = result_df.rename_axis("都道府県名", axis=0)

result_df_2015to2017 = result_df
result_df_2015to2017 

,5_2015,10_2015,2015,5_2016,10_2016,2016,5_2017,10_2017,2017
都道府県名,,,,,,,,,
三重,98156,299876,398032,94827,292171,386998,91472,284691,376163
京都,171163,524309,695472,171652,515055,686707,173055,508835,681890
佐賀,42382,95817,138199,40999,94228,135227,39346,93077,132423
兵庫,340850,1141199,1482049,337457,1117624,1455081,335507,1098087,1433594
北海道,207043,579475,786518,205298,568242,773540,203011,562189,765200
千葉,374682,1392333,1767015,372013,1360011,1732024,370654,1330707,1701361
合計,7119445,22745916,29865361,7055538,22324216,29379754,7234133,22488579,29722712
和歌山,44526,134823,179349,43321,131316,174637,42278,128379,170657
埼玉,445265,1391553,1836818,442704,1359243,1801947,441065,1332009,1773074


In [12]:
# result_df_2018to2022 と result_df_2015to2017 を列「都道府県名」をキーにして結合
merged_df = pd.merge(result_df_2015to2017, result_df_2018to2022, on='都道府県名', suffixes=('_2018to2022', '_2015to2017'))



In [13]:
# '都道府県名'を列として追加する
merged_df['都道府県名'] = merged_df.index

# 都道府県名と'2015'から'2022'までの列名だけを残した別のデータフレームを作成
columns_to_keep = ['都道府県名'] + [str(year) for year in range(2015, 2023)]
time_series_columns_df = merged_df[columns_to_keep]

time_series_columns_df = merged_df.set_index('都道府県名').filter(regex=r'^2015$|^2016$|^2017$|^2018$|^2019$|^2020$|^2021$|^2022$')
time_series_columns_df

time_series_columns_df = time_series_columns_df.reindex(result_df_2018to2022.index)

time_series_columns_df

,2015,2016,2017,2018,2019,2020,2021,2022
都道府県名,,,,,,,,
北海道,786518,773540,765200,"775,934","791,244","722,965","629,747","546,403"
青森,120346,118215,114690,"114,705","115,682","104,044","89,701","76,181"
岩手,129921,126182,121296,"121,026","122,580","109,872","96,109","82,233"
宮城,368230,364138,357765,"359,966","364,270","332,154","293,201","251,974"
秋田,101838,96845,91535,"89,203","89,322","79,980","68,591","57,878"
山形,142872,139542,134767,"133,493","133,568","119,445","102,438","86,411"
福島,270645,267361,259716,"257,729","255,547","228,621","197,226","166,293"
茨城,605571,587452,570440,"566,675","562,859","505,940","438,739","381,233"
栃木,373154,364328,355017,"352,981","352,306","317,601","273,926","236,540"


In [19]:
# 「都道府県名」列を取り出します
prefecture_column = time_series_columns_df['都道府県名']

# 「都道府県名」列をデータフレームから削除します
time_series_columns_df.drop(columns=['都道府県名'], inplace=True)

# 「都道府県名」列をデータフレームの一番左側に挿入します
time_series_columns_df.insert(0, '都道府県名', prefecture_column)

# CSVファイルとして保存
time_series_columns_df.to_csv("passport_time_series_2015-2022.csv", index=False)







### 2014

In [15]:
pdf = pdfplumber.open("data/jpn2014.pdf")
page_jpn_valid = pdf.pages[9]
jpn_valid = page_jpn_valid.extract_table()
pd_jpn_valid = pd.DataFrame(jpn_valid[2:], columns=jpn_valid[1])
pd_jpn_valid

,都道府県名,None,None,5年旅券,10年旅券,合計
0,北海道庁,None,None,"213,454","586,530","799,984"
1,青森県庁,None,None,"40,403","80,845","121,248"
2,岩手県庁,None,None,"47,390","85,419","132,809"
3,宮城県庁,None,None,"102,699","267,429","370,128"
4,秋田県庁,None,None,"36,966","69,704","106,670"
5,山形県庁,None,None,"50,613","95,189","145,802"
6,福島県庁,None,None,"88,362","187,686","276,048"
7,茨城県庁,None,None,"156,631","462,628","619,259"
8,栃木県庁,None,None,"102,430","278,367","380,797"
9,群馬県庁,None,None,"94,984","275,781","370,765"
